<a href="https://colab.research.google.com/github/septembermoon7/Inter-case/blob/main/log_trace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install torch torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.6/890.6 kB 9.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
from torch.utils.data import TensorDataset, random_split, DataLoader
import torch.optim as optim

In [ ]:
from preprocessing import sort_log, debiasing, create_time_features, mapping_case_id, add_soc_eoc
from preprocessing import train_mapping_event_name, test_mapping_event_name, train_standardize, test_standardize
from train_test_split import create_table_without_discard_case, get_train_test_split_point
from create_prefix_suffix import create_log_prefix_tensor, create_trace_prefix_tensor, create_trace_suffix_tensor
from create_encoder_decoder import Encoder, Decoder, Seq2Seq_one_input, Seq2Seq_cat, Seq2Seq_add, Seq2Seq_mul, normalized_DL_distance

# Create training dataloader pipeline
Identical with the steps in the function create_train_valid_dataloader

In [ ]:
# Define parameters
csv_path = 'cases2000.csv'
end_date = '2017-01'
max_duration = 47.81
test_ratio = 0.1
validation_ratio = 0.1
log_prefix_length = 30
trace_prefix_length = 15
trace_suffix_length = 35
num_act = 28 # number of activity labels in training set: 24

In [ ]:
set_name = 'train'
log_col_name = ['concept:name', 'log_ts_pre']
trace_prefix_col_name = ['concept:name', 'trace_ts_pre', 'trace_ts_start']
trace_suffix_col_name = ['concept:name', 'trace_ts_pre']
categorical_features = ['concept:name']
continuous_features = ['log_ts_pre', 'trace_ts_pre', 'trace_ts_start']
case_id = 'case:concept:name'
timestamp = 'time:timestamp'
event_name = 'concept:name'
event_idx = 'event_idx'

In [ ]:
# 1. Tranform csv to dataframe
df = pd.read_csv(csv_path)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79917 entries, 0 to 79916
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         79917 non-null  int64 
 1   case:concept:name  79917 non-null  object
 2   time:timestamp     79917 non-null  object
 3   concept:name       79917 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.4+ MB
None


In [ ]:
print("Number of cases:", df[case_id].nunique())
print("Number of activity labels:", df[event_name].nunique())

Number of cases: 2000
Number of activity labels: 24


In [ ]:
# 2. Debiasing and cleaning
# df = debiasing(df, end_date, max_duration,
#               case_id, timestamp)

In [ ]:
# 3. Sort dataframe by timestamp
df = sort_log(df,
              timestamp)

In [ ]:
# 4. Get rid of discard case
df_no_discard = create_table_without_discard_case(df, test_ratio,
                                                  case_id, timestamp)
print(df_no_discard.info())
print("Number of cases:", df_no_discard[case_id].nunique())
print("Number of activity labels:", df_no_discard[event_name].nunique())

<class 'pandas.core.frame.DataFrame'>
Index: 29908 entries, 0 to 79513
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   Unnamed: 0         29908 non-null  int64              
 1   case:concept:name  29908 non-null  object             
 2   time:timestamp     29908 non-null  datetime64[ns, UTC]
 3   concept:name       29908 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(1), object(2)
memory usage: 1.1+ MB
None
Number of cases: 760
Number of activity labels: 24


In [ ]:
# 5. Subset the dataframe to dataframe before training / test split
train_test_split_time, train_test_split_idx = get_train_test_split_point(df, test_ratio,
                                                      case_id, timestamp)
print(train_test_split_time)
print(train_test_split_idx)

2016-01-26 10:05:22.374000+00:00
48102


In [ ]:
training_df = df_no_discard[df_no_discard[timestamp] < train_test_split_time]
print(training_df.info())
print("Number of cases:", training_df[case_id].nunique())
print("Number of activity labels:", training_df[event_name].nunique())

<class 'pandas.core.frame.DataFrame'>
Index: 21619 entries, 0 to 48028
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   Unnamed: 0         21619 non-null  int64              
 1   case:concept:name  21619 non-null  object             
 2   time:timestamp     21619 non-null  datetime64[ns, UTC]
 3   concept:name       21619 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(1), object(2)
memory usage: 844.5+ KB
None
Number of cases: 560
Number of activity labels: 24


In [ ]:
# 6. Create time features
training_df = create_time_features(training_df, case_id,timestamp)
print(training_df.info())
print(training_df.head(20))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21619 entries, 0 to 21618
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   Unnamed: 0         21619 non-null  int64              
 1   case:concept:name  21619 non-null  object             
 2   time:timestamp     21619 non-null  datetime64[ns, UTC]
 3   concept:name       21619 non-null  object             
 4   log_ts_pre         21619 non-null  float64            
 5   trace_ts_pre       21619 non-null  float64            
 6   trace_ts_start     21619 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(3), int64(1), object(2)
memory usage: 1.2+ MB
None
    Unnamed: 0       case:concept:name                   time:timestamp  \
0            0   Application_652823628 2016-01-01 09:51:15.304000+00:00   
1            1   Application_652823628 2016-01-01 09:51:15.352000+00:00   
2            2   Application_6528236

In [ ]:
# 7. Standardize time features
training_df, mean_dict, std_dict = train_standardize(training_df,
                                                     continuous_features)
print(training_df.head(20))
print(mean_dict)
print(std_dict)

    Unnamed: 0       case:concept:name                   time:timestamp  \
0            0   Application_652823628 2016-01-01 09:51:15.304000+00:00   
1            1   Application_652823628 2016-01-01 09:51:15.352000+00:00   
2            2   Application_652823628 2016-01-01 09:51:15.774000+00:00   
3            3   Application_652823628 2016-01-01 09:52:36.392000+00:00   
4            4   Application_652823628 2016-01-01 09:52:36.403000+00:00   
5            5   Application_652823628 2016-01-01 09:52:36.413000+00:00   
6            6  Application_1691306052 2016-01-01 10:16:11.500000+00:00   
7            7  Application_1691306052 2016-01-01 10:16:11.549000+00:00   
8            8  Application_1691306052 2016-01-01 10:16:11.740000+00:00   
9            9  Application_1691306052 2016-01-01 10:17:31.573000+00:00   
10          10  Application_1691306052 2016-01-01 10:17:31.584000+00:00   
11          11  Application_1691306052 2016-01-01 10:17:31.594000+00:00   
12          12   Applicat

In [ ]:
# 8. Map case ID to numbers
training_df, case_id_dict= mapping_case_id(training_df,
                                     case_id)
print(len(case_id_dict))
print(training_df.head(20))

560
    Unnamed: 0  case:concept:name                   time:timestamp  \
0            0                  1 2016-01-01 09:51:15.304000+00:00   
1            1                  1 2016-01-01 09:51:15.352000+00:00   
2            2                  1 2016-01-01 09:51:15.774000+00:00   
3            3                  1 2016-01-01 09:52:36.392000+00:00   
4            4                  1 2016-01-01 09:52:36.403000+00:00   
5            5                  1 2016-01-01 09:52:36.413000+00:00   
6            6                  2 2016-01-01 10:16:11.500000+00:00   
7            7                  2 2016-01-01 10:16:11.549000+00:00   
8            8                  2 2016-01-01 10:16:11.740000+00:00   
9            9                  2 2016-01-01 10:17:31.573000+00:00   
10          10                  2 2016-01-01 10:17:31.584000+00:00   
11          11                  2 2016-01-01 10:17:31.594000+00:00   
12          12                  3 2016-01-01 11:19:38.177000+00:00   
13          13  

In [ ]:
# 9. Insert SOC and EOC rows
training_df = add_soc_eoc(training_df,
                          case_id, timestamp, event_name)
print(training_df.info())
print(training_df.head(20))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22739 entries, 0 to 22738
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   Unnamed: 0         22739 non-null  int64              
 1   case:concept:name  22739 non-null  int64              
 2   time:timestamp     22739 non-null  datetime64[ns, UTC]
 3   concept:name       22739 non-null  object             
 4   log_ts_pre         22739 non-null  float64            
 5   trace_ts_pre       22739 non-null  float64            
 6   trace_ts_start     22739 non-null  float64            
 7   event_idx          22739 non-null  int64              
dtypes: datetime64[ns, UTC](1), float64(3), int64(3), object(1)
memory usage: 1.4+ MB
None
    Unnamed: 0  case:concept:name                   time:timestamp  \
0            0                  1 2016-01-01 09:51:15.304000+00:00   
1            0                  1 2016-01-01 09:51:15.304000+

In [ ]:
# 10. Mapping event name to numbers
training_df, train_event_name_dict = train_mapping_event_name(training_df,
                                                                  event_name)
print(training_df.head(20))
print(train_event_name_dict)

    Unnamed: 0  case:concept:name                   time:timestamp  \
0            0                  1 2016-01-01 09:51:15.304000+00:00   
1            0                  1 2016-01-01 09:51:15.304000+00:00   
2            1                  1 2016-01-01 09:51:15.352000+00:00   
3            2                  1 2016-01-01 09:51:15.774000+00:00   
4            3                  1 2016-01-01 09:52:36.392000+00:00   
5            4                  1 2016-01-01 09:52:36.403000+00:00   
6            5                  1 2016-01-01 09:52:36.413000+00:00   
7            6                  2 2016-01-01 10:16:11.500000+00:00   
8            6                  2 2016-01-01 10:16:11.500000+00:00   
9            7                  2 2016-01-01 10:16:11.549000+00:00   
10           8                  2 2016-01-01 10:16:11.740000+00:00   
11           9                  2 2016-01-01 10:17:31.573000+00:00   
12          10                  2 2016-01-01 10:17:31.584000+00:00   
13          11      

In [ ]:
# Set printed tensor format
torch.set_printoptions(sci_mode=False, precision=2)

In [ ]:
# 11. Create train_log_prefix_tensor, train_trace_prefix_tensor, train_suffix_act_tensor, train_suffix_time_tensor
train_log_prefix_tensor = create_log_prefix_tensor(training_df, log_prefix_length, set_name, test_ratio, num_act,
                                                       log_col_name, categorical_features, case_id, timestamp, event_name)
print(train_log_prefix_tensor.shape)
print(train_log_prefix_tensor[:5])

torch.Size([22179, 30, 29])
tensor([[[     0.00,      0.00,      0.00,  ...,      0.00,      0.00,
          -10000.00],
         [     0.00,      0.00,      0.00,  ...,      0.00,      0.00,
          -10000.00],
         [     0.00,      0.00,      0.00,  ...,      0.00,      0.00,
          -10000.00],
         ...,
         [     0.00,      0.00,      0.00,  ...,      0.00,      0.00,
          -10000.00],
         [     0.00,      0.00,      0.00,  ...,      0.00,      0.00,
          -10000.00],
         [     0.00,      0.00,      1.00,  ...,      0.00,      0.00,
              -0.07]],

        [[     0.00,      0.00,      0.00,  ...,      0.00,      0.00,
          -10000.00],
         [     0.00,      0.00,      0.00,  ...,      0.00,      0.00,
          -10000.00],
         [     0.00,      0.00,      0.00,  ...,      0.00,      0.00,
          -10000.00],
         ...,
         [     0.00,      0.00,      0.00,  ...,      0.00,      0.00,
          -10000.00],
         [  

In [ ]:
train_trace_prefix_tensor = create_trace_prefix_tensor(training_df, trace_prefix_length, set_name, test_ratio, num_act,
                                                           trace_prefix_col_name, categorical_features, event_idx, case_id, timestamp, event_name)
print(train_trace_prefix_tensor.shape)
print(train_trace_prefix_tensor[:5])

torch.Size([22179, 15, 30])
tensor([[[     0.00,      0.00,      0.00,  ...,      0.00, -10000.00,
          -10000.00],
         [     0.00,      0.00,      0.00,  ...,      0.00, -10000.00,
          -10000.00],
         [     0.00,      0.00,      0.00,  ...,      0.00, -10000.00,
          -10000.00],
         ...,
         [     0.00,      0.00,      0.00,  ...,      0.00, -10000.00,
          -10000.00],
         [     0.00,      0.00,      0.00,  ...,      0.00, -10000.00,
          -10000.00],
         [     0.00,      0.00,      1.00,  ...,      0.00,     -0.27,
              -1.05]],

        [[     0.00,      0.00,      0.00,  ...,      0.00, -10000.00,
          -10000.00],
         [     0.00,      0.00,      0.00,  ...,      0.00, -10000.00,
          -10000.00],
         [     0.00,      0.00,      0.00,  ...,      0.00, -10000.00,
          -10000.00],
         ...,
         [     0.00,      0.00,      0.00,  ...,      0.00, -10000.00,
          -10000.00],
         [  

In [ ]:
train_suffix_act_tensor, train_suffix_time_tensor = create_trace_suffix_tensor(training_df, trace_suffix_length, set_name, test_ratio,
                                                                                   trace_suffix_col_name, categorical_features, event_idx, case_id, timestamp, event_name)
print(train_suffix_act_tensor.shape)
print(train_suffix_act_tensor[:5])


torch.Size([22179, 35])
tensor([[ 4,  5,  6,  6,  7,  8,  7,  7,  9, 10, 11, 12,  7, 13, 13, 14, 13, 13,
         13, 13, 17, 17, 18, 19, 17, 17, 20, 20, 21, 20, 20, 20, 20, 17, 17],
        [ 5,  6,  6,  7,  8,  7,  7,  9, 10, 11, 12,  7, 13, 13, 14, 13, 13, 13,
         13, 17, 17, 18, 19, 17, 17, 20, 20, 21, 20, 20, 20, 20, 17, 17, 18],
        [ 6,  6,  7,  8,  7,  7,  9, 10, 11, 12,  7, 13, 13, 14, 13, 13, 13, 13,
         17, 17, 18, 19, 17, 17, 20, 20, 21, 20, 20, 20, 20, 17, 17, 18, 17],
        [ 6,  7,  8,  7,  7,  9, 10, 11, 12,  7, 13, 13, 14, 13, 13, 13, 13, 17,
         17, 18, 19, 17, 17, 20, 20, 21, 20, 20, 20, 20, 17, 17, 18, 17, 23],
        [ 7,  8,  7,  7,  9, 10, 11, 12,  7, 13, 13, 14, 13, 13, 13, 13, 17, 17,
         18, 19, 17, 17, 20, 20, 21, 20, 20, 20, 20, 17, 17, 18, 17, 23, 24]])


In [ ]:
print(train_suffix_time_tensor.shape)
print(train_suffix_time_tensor[:5])

torch.Size([22179, 35])
tensor([[-0.27, -0.27, -0.27, -0.27, -0.27, -0.27,  0.81, -0.26, -0.24, -0.26,
         -0.27, -0.27, -0.27, -0.27, -0.27, -0.27, -0.26,  3.80, -0.27,  7.18,
         -0.27, -0.27, -0.27, -0.27, -0.26,  0.60, -0.27, -0.27, -0.27, -0.27,
         -0.17, -0.26, -0.17, -0.27, -0.27],
        [-0.27, -0.27, -0.27, -0.27, -0.27,  0.81, -0.26, -0.24, -0.26, -0.27,
         -0.27, -0.27, -0.27, -0.27, -0.27, -0.26,  3.80, -0.27,  7.18, -0.27,
         -0.27, -0.27, -0.27, -0.26,  0.60, -0.27, -0.27, -0.27, -0.27, -0.17,
         -0.26, -0.17, -0.27, -0.27, -0.27],
        [-0.27, -0.27, -0.27, -0.27,  0.81, -0.26, -0.24, -0.26, -0.27, -0.27,
         -0.27, -0.27, -0.27, -0.27, -0.26,  3.80, -0.27,  7.18, -0.27, -0.27,
         -0.27, -0.27, -0.26,  0.60, -0.27, -0.27, -0.27, -0.27, -0.17, -0.26,
         -0.17, -0.27, -0.27, -0.27, -0.26],
        [-0.27, -0.27, -0.27,  0.81, -0.26, -0.24, -0.26, -0.27, -0.27, -0.27,
         -0.27, -0.27, -0.27, -0.26,  3.80, -0.27, 

In [ ]:
# 12. Build TensorDataset
train_dataset = TensorDataset(train_log_prefix_tensor, train_trace_prefix_tensor, train_suffix_act_tensor, train_suffix_time_tensor)

In [ ]:
# 13. Split TensorDataset into train_dataset, valid_dataset
train_dataset, valid_dataset = random_split(train_dataset, [1-validation_ratio, validation_ratio])
print(len(train_dataset))
print(len(valid_dataset))

19962
2217


In [ ]:
# 14. Build train_dataloader, valid_dataloader
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=64, shuffle=True)

# Create test dataloader pipeline

In [ ]:
# Define parameters
csv_path = 'cases2000.csv'
end_date = '2017-01'
max_duration = 47.81
test_ratio = 0.1
log_prefix_length = 30
trace_prefix_length = 15
trace_suffix_length = 35
num_act = 28 # number of activity labels in training set: 24

In [ ]:
set_name = 'test'
log_col_name = ['concept:name', 'log_ts_pre']
trace_prefix_col_name = ['concept:name', 'trace_ts_pre', 'trace_ts_start']
trace_suffix_col_name = ['concept:name', 'trace_ts_pre']
categorical_features = ['concept:name']
continuous_features = ['log_ts_pre', 'trace_ts_pre', 'trace_ts_start']
case_id = 'case:concept:name'
timestamp = 'time:timestamp'
event_name = 'concept:name'
event_idx = 'event_idx'

In [ ]:
train_event_name_dict = train_event_name_dict
mean_dict = mean_dict
std_dict = std_dict

In [ ]:
# 1. Tranform csv to dataframe
df = pd.read_csv(csv_path)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79917 entries, 0 to 79916
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         79917 non-null  int64 
 1   case:concept:name  79917 non-null  object
 2   time:timestamp     79917 non-null  object
 3   concept:name       79917 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.4+ MB
None


In [ ]:
# # 2. Debiasing and cleaning
# df = debiasing(df,end_date, max_duration,
#                    case_id, timestamp)

In [ ]:
# 3. Sort dataframe by timestamp
df = sort_log(df,
              timestamp)

In [ ]:
# 4. Create time features
df = create_time_features(df,
                          case_id,timestamp)
print(df.info())
print(df.head(20))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79917 entries, 0 to 79916
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   Unnamed: 0         79917 non-null  int64              
 1   case:concept:name  79917 non-null  object             
 2   time:timestamp     79917 non-null  datetime64[ns, UTC]
 3   concept:name       79917 non-null  object             
 4   log_ts_pre         79917 non-null  float64            
 5   trace_ts_pre       79917 non-null  float64            
 6   trace_ts_start     79917 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(3), int64(1), object(2)
memory usage: 4.3+ MB
None
    Unnamed: 0       case:concept:name                   time:timestamp  \
0            0   Application_652823628 2016-01-01 09:51:15.304000+00:00   
1            1   Application_652823628 2016-01-01 09:51:15.352000+00:00   
2            2   Application_6528236

In [ ]:
# 5. Standardize time features
df = test_standardize(df,
                      mean_dict, std_dict, continuous_features)
print(df.head(20))

    Unnamed: 0       case:concept:name                   time:timestamp  \
0            0   Application_652823628 2016-01-01 09:51:15.304000+00:00   
1            1   Application_652823628 2016-01-01 09:51:15.352000+00:00   
2            2   Application_652823628 2016-01-01 09:51:15.774000+00:00   
3            3   Application_652823628 2016-01-01 09:52:36.392000+00:00   
4            4   Application_652823628 2016-01-01 09:52:36.403000+00:00   
5            5   Application_652823628 2016-01-01 09:52:36.413000+00:00   
6            6  Application_1691306052 2016-01-01 10:16:11.500000+00:00   
7            7  Application_1691306052 2016-01-01 10:16:11.549000+00:00   
8            8  Application_1691306052 2016-01-01 10:16:11.740000+00:00   
9            9  Application_1691306052 2016-01-01 10:17:31.573000+00:00   
10          10  Application_1691306052 2016-01-01 10:17:31.584000+00:00   
11          11  Application_1691306052 2016-01-01 10:17:31.594000+00:00   
12          12   Applicat

In [ ]:
# 6. Map case ID to numbers
df, case_id_dict = mapping_case_id(df,
                        case_id)
print(len(case_id_dict))
print(df.head(20))

2000
    Unnamed: 0  case:concept:name                   time:timestamp  \
0            0                  1 2016-01-01 09:51:15.304000+00:00   
1            1                  1 2016-01-01 09:51:15.352000+00:00   
2            2                  1 2016-01-01 09:51:15.774000+00:00   
3            3                  1 2016-01-01 09:52:36.392000+00:00   
4            4                  1 2016-01-01 09:52:36.403000+00:00   
5            5                  1 2016-01-01 09:52:36.413000+00:00   
6            6                  2 2016-01-01 10:16:11.500000+00:00   
7            7                  2 2016-01-01 10:16:11.549000+00:00   
8            8                  2 2016-01-01 10:16:11.740000+00:00   
9            9                  2 2016-01-01 10:17:31.573000+00:00   
10          10                  2 2016-01-01 10:17:31.584000+00:00   
11          11                  2 2016-01-01 10:17:31.594000+00:00   
12          12                  3 2016-01-01 11:19:38.177000+00:00   
13          13 

In [ ]:
# 7. Insert SOC and EOC rows
df = add_soc_eoc(df,
                  case_id, timestamp, event_name)
print(df.info())
print(df.head(20))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83917 entries, 0 to 83916
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   Unnamed: 0         83917 non-null  int64              
 1   case:concept:name  83917 non-null  int64              
 2   time:timestamp     83917 non-null  datetime64[ns, UTC]
 3   concept:name       83917 non-null  object             
 4   log_ts_pre         83917 non-null  float64            
 5   trace_ts_pre       83917 non-null  float64            
 6   trace_ts_start     83917 non-null  float64            
 7   event_idx          83917 non-null  int64              
dtypes: datetime64[ns, UTC](1), float64(3), int64(3), object(1)
memory usage: 5.1+ MB
None
    Unnamed: 0  case:concept:name                   time:timestamp  \
0            0                  1 2016-01-01 09:51:15.304000+00:00   
1            0                  1 2016-01-01 09:51:15.304000+

In [ ]:
# 8. Mapping event name to numbers
df, test_event_name_dict = test_mapping_event_name(df, train_event_name_dict,
                                                  event_name)
print(df.head(20))
print(test_event_name_dict)

    Unnamed: 0  case:concept:name                   time:timestamp  \
0            0                  1 2016-01-01 09:51:15.304000+00:00   
1            0                  1 2016-01-01 09:51:15.304000+00:00   
2            1                  1 2016-01-01 09:51:15.352000+00:00   
3            2                  1 2016-01-01 09:51:15.774000+00:00   
4            3                  1 2016-01-01 09:52:36.392000+00:00   
5            4                  1 2016-01-01 09:52:36.403000+00:00   
6            5                  1 2016-01-01 09:52:36.413000+00:00   
7            6                  2 2016-01-01 10:16:11.500000+00:00   
8            6                  2 2016-01-01 10:16:11.500000+00:00   
9            7                  2 2016-01-01 10:16:11.549000+00:00   
10           8                  2 2016-01-01 10:16:11.740000+00:00   
11           9                  2 2016-01-01 10:17:31.573000+00:00   
12          10                  2 2016-01-01 10:17:31.584000+00:00   
13          11      

In [ ]:
_, split_idx = get_train_test_split_point(df, test_ratio,
                                                case_id=case_id, timestamp=timestamp)
print(split_idx)
print(df[split_idx-20:split_idx])
print(df[split_idx:split_idx+20])

50462
       Unnamed: 0  case:concept:name                   time:timestamp  \
50442       48082               1363 2016-01-26 10:02:26.587000+00:00   
50443       48083               1363 2016-01-26 10:02:26.599000+00:00   
50444       48084               1363 2016-01-26 10:02:26.603000+00:00   
50445       48085               1363 2016-01-26 10:02:27.556000+00:00   
50446       48086                856 2016-01-26 10:03:02.691000+00:00   
50447       48087                856 2016-01-26 10:03:02.697000+00:00   
50448       48088                856 2016-01-26 10:03:02.699000+00:00   
50449       48089                856 2016-01-26 10:03:03.578000+00:00   
50450       48090                856 2016-01-26 10:03:26.108000+00:00   
50451       48091               1363 2016-01-26 10:03:27.015000+00:00   
50452       48092               1792 2016-01-26 10:03:35.401000+00:00   
50453       48093               1800 2016-01-26 10:03:36.964000+00:00   
50454       48094               1800 2016-01-

In [ ]:
# 9. Create test_log_prefix_tensor, test_trace_prefix_tensor, test_suffix_act_tensor, test_suffix_time_tensor
test_log_prefix_tensor = create_log_prefix_tensor(df, log_prefix_length, set_name, test_ratio, num_act,
                                                    log_col_name, categorical_features, case_id, timestamp, event_name)
print(test_log_prefix_tensor.shape)
print(test_log_prefix_tensor[:5])

torch.Size([32015, 30, 29])
tensor([[[ 0.00,  0.00,  0.00,  ...,  0.00,  0.00, -0.07],
         [ 0.00,  0.00,  0.00,  ...,  0.00,  0.00, -0.06],
         [ 0.00,  0.00,  0.00,  ...,  0.00,  0.00, -0.05],
         ...,
         [ 0.00,  0.00,  0.00,  ...,  0.00,  0.00, -0.05],
         [ 0.00,  0.00,  0.00,  ...,  0.00,  0.00, -0.06],
         [ 0.00,  0.00,  1.00,  ...,  0.00,  0.00, -0.02]],

        [[ 0.00,  0.00,  0.00,  ...,  0.00,  0.00, -0.06],
         [ 0.00,  0.00,  0.00,  ...,  0.00,  0.00, -0.05],
         [ 0.00,  0.00,  0.00,  ...,  0.00,  0.00, -0.07],
         ...,
         [ 0.00,  0.00,  0.00,  ...,  0.00,  0.00, -0.06],
         [ 0.00,  0.00,  1.00,  ...,  0.00,  0.00, -0.02],
         [ 0.00,  0.00,  0.00,  ...,  0.00,  0.00, -0.02]],

        [[ 0.00,  0.00,  0.00,  ...,  0.00,  0.00, -0.05],
         [ 0.00,  0.00,  0.00,  ...,  0.00,  0.00, -0.07],
         [ 0.00,  0.00,  0.00,  ...,  0.00,  0.00, -0.06],
         ...,
         [ 0.00,  0.00,  1.00,  ...,  0.0

In [ ]:
test_trace_prefix_tensor = create_trace_prefix_tensor(df, trace_prefix_length, set_name, test_ratio, num_act,
                                                      trace_prefix_col_name, categorical_features, event_idx, case_id, timestamp, event_name)
print(test_trace_prefix_tensor.shape)
print(test_trace_prefix_tensor[:5])

torch.Size([32015, 15, 30])
tensor([[[     0.00,      0.00,      0.00,  ...,      0.00, -10000.00,
          -10000.00],
         [     0.00,      0.00,      0.00,  ...,      0.00, -10000.00,
          -10000.00],
         [     0.00,      0.00,      0.00,  ...,      0.00, -10000.00,
          -10000.00],
         ...,
         [     0.00,      0.00,      0.00,  ...,      0.00, -10000.00,
          -10000.00],
         [     0.00,      0.00,      0.00,  ...,      0.00, -10000.00,
          -10000.00],
         [     0.00,      0.00,      1.00,  ...,      0.00,     -0.27,
              -1.05]],

        [[     0.00,      0.00,      0.00,  ...,      0.00, -10000.00,
          -10000.00],
         [     0.00,      0.00,      0.00,  ...,      0.00, -10000.00,
          -10000.00],
         [     0.00,      0.00,      0.00,  ...,      0.00, -10000.00,
          -10000.00],
         ...,
         [     0.00,      0.00,      0.00,  ...,      0.00, -10000.00,
          -10000.00],
         [  

In [ ]:
test_suffix_act_tensor, test_suffix_time_tensor = create_trace_suffix_tensor(df, trace_suffix_length, set_name, test_ratio,
                                                                                trace_suffix_col_name, categorical_features, event_idx, case_id, timestamp, event_name)
print(test_suffix_act_tensor.shape)
print(test_suffix_act_tensor[:5])

torch.Size([32015, 35])
tensor([[ 4,  5,  6,  6,  7,  8,  7,  9, 10, 11, 12,  7, 13, 13, 14, 13, 13, 13,
         13, 17, 17, 18, 19, 17, 17, 17, 17, 17, 17, 17, 20, 20, 21, 20, 20],
        [ 5,  6,  6,  7,  8,  7,  9, 10, 11, 12,  7, 13, 13, 14, 13, 13, 13, 13,
         17, 17, 18, 19, 17, 17, 17, 17, 17, 17, 17, 20, 20, 21, 20, 20, 20],
        [ 6,  6,  7,  8,  7,  9, 10, 11, 12,  7, 13, 13, 14, 13, 13, 13, 13, 17,
         17, 18, 19, 17, 17, 17, 17, 17, 17, 17, 20, 20, 21, 20, 20, 20, 20],
        [ 6,  7,  8,  7,  9, 10, 11, 12,  7, 13, 13, 14, 13, 13, 13, 13, 17, 17,
         18, 19, 17, 17, 17, 17, 17, 17, 17, 20, 20, 21, 20, 20, 20, 20, 17],
        [26,  3, 17,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])


In [ ]:
print(test_suffix_time_tensor.shape)
print(test_suffix_time_tensor[:5])

torch.Size([32015, 35])
tensor([[    -0.27,     -0.27,     -0.27,     -0.27,     -0.27,     -0.27,
             -0.24,     -0.26,     -0.26,     -0.27,     -0.26,     -0.27,
             -0.27,     -0.27,     -0.27,     -0.27,      3.84,     -0.27,
              1.84,     -0.27,     -0.27,     -0.27,     -0.27,     -0.26,
              1.66,     -0.27,     -0.10,     -0.26,     -0.16,     -0.26,
             -0.27,     -0.27,     -0.27,     -0.27,     -0.08],
        [    -0.27,     -0.27,     -0.27,     -0.27,     -0.27,     -0.24,
             -0.26,     -0.26,     -0.27,     -0.26,     -0.27,     -0.27,
             -0.27,     -0.27,     -0.27,      3.84,     -0.27,      1.84,
             -0.27,     -0.27,     -0.27,     -0.27,     -0.26,      1.66,
             -0.27,     -0.10,     -0.26,     -0.16,     -0.26,     -0.27,
             -0.27,     -0.27,     -0.27,     -0.08,     -0.26],
        [    -0.27,     -0.27,     -0.27,     -0.27,     -0.24,     -0.26,
             -0.26,  

In [ ]:
# 10. Build TensorDataset
test_dataset = TensorDataset(test_log_prefix_tensor, test_trace_prefix_tensor, test_suffix_act_tensor, test_suffix_time_tensor)
print(len(test_dataset))

32015


In [ ]:
# 11. Build test_dataloader, valid_dataloader
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# Run experiment

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_act = 28

learning_rate = 0.01

num_epochs = 10

num_layers = 4

enc_hidden_size = 50
log_enc_input_size = num_act + 1
trace_enc_input_size = num_act + 2

dec_hidden_size = 50 # For seq2seq log , seq2seq trace, seq2seq add, seq2seq mul
dec_cat_hidden_size = 100 # For seq2seq cat

act_dec_input_size = num_act
act_dec_output_size = num_act

time_dec_input_size = 1
time_dec_output_size = 1

In [ ]:
log_encoder = Encoder(input_size=log_enc_input_size,
                      hidden_size=enc_hidden_size,
                      num_layers=num_layers).to(device)

trace_encoder = Encoder(input_size=trace_enc_input_size,
                        hidden_size=enc_hidden_size,
                        num_layers=num_layers).to(device)

act_decoder = Decoder(input_size=act_dec_input_size,
                      hidden_size=dec_hidden_size,
                      output_size=act_dec_output_size,
                      num_layers=num_layers).to(device)

time_decoder = Decoder(input_size=time_dec_input_size,
                      hidden_size=dec_hidden_size,
                      output_size=time_dec_output_size,
                      num_layers=num_layers).to(device)

act_cat_decoder = Decoder(input_size=act_dec_input_size,
                      hidden_size=dec_cat_hidden_size,
                      output_size=act_dec_output_size,
                      num_layers=num_layers).to(device)

time_cat_decoder = Decoder(input_size=time_dec_input_size,
                      hidden_size=dec_cat_hidden_size,
                      output_size=time_dec_output_size,
                      num_layers=num_layers).to(device)

In [ ]:
def training(model, dataloader, teacher_forcing_ratio=0.5):

    model.train()

    # Define optimizer
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Define loss function
    act_criterion = nn.CrossEntropyLoss(ignore_index=0)
    time_criterion = nn.L1Loss()

    for epoch in range(num_epochs):

        epoch_loss = 0

        for batch in dataloader:

            train_log_prefix, train_trace_prefix, train_suffix_act, train_suffix_time = batch
            train_log_prefix = train_log_prefix.float().to(device)
            train_trace_prefix = train_trace_prefix.float().to(device)
            train_suffix_act = train_suffix_act.to(torch.long).to(device)
            train_suffix_time = train_suffix_time.float().to(device)
            # train_log_prefix: batch_size, log_prefix_len, num_act + 1
            # train_trace_prefix: batch_size, trace_prefix_len, num_act + 2
            # train_suffix_act: batch_size, suffix_len
            # train_suffi_time: batch_size, suffix_len

            optimizer.zero_grad()

            if model == model_log:
                act_predictions, time_predictions = model(train_log_prefix, train_suffix_act, train_suffix_time, teacher_forcing_ratio)

            elif model == model_trace:
                act_predictions, time_predictions = model(train_trace_prefix, train_suffix_act, train_suffix_time, teacher_forcing_ratio)

            else:
                act_predictions, time_predictions = model(train_log_prefix, train_trace_prefix, train_suffix_act, train_suffix_time, teacher_forcing_ratio)

            act_predictions = act_predictions.to(device)
            # act_predictions shape: batch_size, suffix_len, num_act
            time_predictions = time_predictions.to(device)
            # time_predictions shape: batch_size, suffix_len, 1

            # Mask time padding
            train_suffix_time = train_suffix_time.unsqueeze(-1)
            # train_suffix_time shape: batch_size, suffix_len, 1
            mask = (train_suffix_time != -10000).to(device)
            masked_train_suffix_time = torch.masked_select(train_suffix_time, mask) # The result is a 1D tensor
            masked_time_predictions = torch.masked_select(time_predictions, mask) # The result is a 1D tensor

            # Reshape act_predictions and train_suffix_act
            act_predictions = act_predictions.view(-1, act_predictions.size(-1))  # Shape: (batch_size * seq_length, num_classes)
            train_suffix_act = train_suffix_act.view(-1)  # Shape: (batch_size * seq_length)

            act_loss = act_criterion(act_predictions, train_suffix_act)
            time_loss = time_criterion(masked_time_predictions, masked_train_suffix_time)

            loss = 0.5 * act_loss + 0.5 * time_loss

            loss.backward()

            optimizer.step()

            epoch_loss += loss.item()

        avg_epoch_loss = epoch_loss / len(dataloader)
        if (epoch + 1) % 10 == 0:
          print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_epoch_loss:.4f}")

In [ ]:
def evaluation(model, dataloader, teacher_forcing_ratio=0.5):

    model.eval()
    epoch_loss = 0

    time_criterion = nn.L1Loss()

    with torch.no_grad():

        for batch in dataloader:

            test_log_prefix, test_trace_prefix, test_suffix_act, test_suffix_time = batch
            test_log_prefix = test_log_prefix.float().to(device)
            test_trace_prefix = test_trace_prefix.float().to(device)
            test_suffix_act = test_suffix_act.to(torch.long).to(device)
            test_suffix_time = test_suffix_time.float().to(device)
            # test_log_prefix: batch_size, log_prefix_len, num_act + 1
            # test_trace_prefix: batch_size, trace_prefix_len, num_act + 2
            # test_suffix_act: batch_size, suffix_len
            # test_suffi_time: batch_size, suffix_len

            if model == model_log:
                act_predictions, time_predictions = model(test_log_prefix, test_suffix_act, test_suffix_time, teacher_forcing_ratio)

            elif model == model_trace:
                act_predictions, time_predictions = model(test_trace_prefix, test_suffix_act, test_suffix_time, teacher_forcing_ratio)

            else:
                act_predictions, time_predictions = model(test_log_prefix, test_trace_prefix, test_suffix_act, test_suffix_time, teacher_forcing_ratio)

            act_predictions = act_predictions.to(device)
            # act_predictions shape: batch_size, suffix_len, num_act
            time_predictions = time_predictions.to(device)
            # time_predictions shape: batch_size, suffix_len, 1

            # Mask time padding
            test_suffix_time = test_suffix_time.unsqueeze(-1)
            # train_suffix_time shape: batch_size, suffix_len, 1
            mask = (test_suffix_time != -10000).to(device)
            masked_test_suffix_time = torch.masked_select(test_suffix_time, mask) # The result is a 1D tensor
            masked_time_predictions = torch.masked_select(time_predictions, mask) # The result is a 1D tensor

            act_loss = normalized_DL_distance(act_predictions, test_suffix_act)
            time_loss = time_criterion(masked_time_predictions, masked_test_suffix_time)

            loss = 0.5 * act_loss + 0.5 * time_loss

            epoch_loss += loss.item()

        return epoch_loss / len(dataloader)

In [ ]:
model_log = Seq2Seq_one_input(num_act=num_act,
                              encoder=log_encoder,
                              act_decoder=act_decoder,
                              time_decoder=time_decoder).to(device)

In [ ]:
training(model_log, train_dataloader)

Epoch 10/10, Average Loss: 0.5055


In [ ]:
torch.save(model_log.state_dict(), 'model_log_checkpoint.pth')

In [ ]:
model_log.load_state_dict(torch.load('model_log_checkpoint.pth'))

In [ ]:
test_loss = evaluation(model_log, test_dataloader)
print(test_loss)

0.4196525403482471


In [ ]:
model_trace = Seq2Seq_one_input(num_act=num_act,
                                encoder=trace_encoder,
                                act_decoder=act_decoder,
                                time_decoder=time_decoder).to(device)

In [ ]:
training(model_trace, train_dataloader)

Epoch 10/10, Average Loss: 0.4750


In [ ]:
torch.save(model_trace.state_dict(), 'model_trace_checkpoint.pth')

In [ ]:
model_trace.load_state_dict(torch.load('model_trace_checkpoint.pth'))

<ipython-input-55-f3af29437025>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_trace.load_state_dict(torch.load('model_trace_checkpoint.pth'))


<All keys matched successfully>

In [ ]:
test_loss = evaluation(model_trace, test_dataloader)
print(test_loss)

0.42727932768191645


In [ ]:
model_cat = Seq2Seq_cat(num_act=num_act,
                        log_encoder=log_encoder,
                        trace_encoder=trace_encoder,
                        act_cat_decoder=act_cat_decoder,
                        time_cat_decoder=time_cat_decoder).to(device)

In [ ]:
training(model_cat, train_dataloader)

Epoch 10/10, Average Loss: 0.4717


In [ ]:
torch.save(model_cat.state_dict(), 'model_cat_checkpoint.pth')

In [ ]:
model_cat.load_state_dict(torch.load('model_cat_checkpoint.pth'))

<ipython-input-60-64ff29b293cb>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_cat.load_state_dict(torch.load('model_cat_checkpoint.pth'))


<All keys matched successfully>

In [ ]:
test_loss = evaluation(model_cat, test_dataloader)
print(test_loss)

0.4245563301140677


In [ ]:
model_add = Seq2Seq_add(num_act=num_act,
                        log_encoder=log_encoder,
                        trace_encoder=trace_encoder,
                        act_decoder=act_decoder,
                        time_decoder=time_decoder).to(device)

In [ ]:
training(model_add, train_dataloader)

Epoch 10/10, Average Loss: 0.4640


In [ ]:
test_loss = evaluation(model_add, test_dataloader)
print(test_loss)

0.41982997843843256


In [ ]:
model_mul = Seq2Seq_mul(num_act=num_act,
                        log_encoder=log_encoder,
                        trace_encoder=trace_encoder,
                        act_decoder=act_decoder,
                        time_decoder=time_decoder).to(device)

In [ ]:
training(model_mul, train_dataloader)

Epoch 10/10, Average Loss: 0.4653


In [ ]:
torch.save(model_mul.state_dict(), 'model_mul_checkpoint.pth')

In [ ]:
model_mul.load_state_dict(torch.load('model_mul_checkpoint.pth'))

<ipython-input-69-bce2518012ae>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_mul.load_state_dict(torch.load('model_mul_checkpoint.pth'))


<All keys matched successfully>

In [ ]:
test_loss = evaluation(model_mul, test_dataloader)
print(test_loss)

0.42536263283855186
